In [ ]:
%pip install --upgrade --quiet docx2txt langchain-community
# text spliter
%pip install -U langchain-text-splitters
# chroma 사용
%pip install langchain-chroma
# RetrievalQA Chain -> Q&A
# langchainhub 프롬프트 다수 제공
%pip install -U langchain langchainhub --quiet

In [5]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 하나의 청크가 가질 수 있는 토큰 수
    chunk_overlap=200, # 겹쳐서 자르기 오버랩(유사도 검색시 범위를 늘려줌줌)
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [6]:
# 실행 테스트
# document_list

In [9]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
from langchain_chroma import Chroma

# embedding 한 내용을 바탕으로 쪼개서 저장
# chroma는 인메모리 
# database = Chroma.from_documents(documents=document_list, embedding=embedding)

# persist_directory 활용
# collection_name : 테이블 네임
# persist_directory : 저장 결과 -> 남아있게 된다
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")

# 이미 저장된 데이터를 사용할 때 
# database = Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding)

In [ ]:
query='연봉 5천만원인 직장인의 소득세는?'

# 유사도 검색
# similarity_search(query, k) : k는 유사도 검색 결과 개수
retrieve_docs = database.similarity_search(query)

In [ ]:
# 이제 검색을 하자
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model='gpt-4o')

# prompt= f"""
# RTF에 맞춰서 프롬프팅
# """

# gpt_msg = llm.invoke(prompt)
# gpt_msg.content

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o')

from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    # 값 가져오기
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
ai_message = qa_chain.invoke({"query":query})
ai_message